In [1]:
import re
import gzip
import json
from glob import glob

import pandas as pd
from bs4 import BeautifulSoup
from pymystem3 import Mystem
from many_stop_words import get_stop_words

In [2]:
code = pd.read_csv('tmp/criminal_code.csv')

In [3]:
with gzip.GzipFile('data/court_orders.json.gz', 'r') as f:
    df = pd.DataFrame(list(map(lambda line: json.loads(line), f)))

In [4]:
df['title'] = df['title'].map(lambda x: x[0])
df['body']= df['body'].map(lambda x: x[0])

In [5]:
def parse_table(table_body):
    data = dict()
    for line in BeautifulSoup(table_body, 'lxml').find_all('tr'):
        data[line.find('td').get_text()] = line.find('th').get_text()
    return data

Выкидываем записи с пустой таблицей метаданных:

In [6]:
df = df.drop(df[df['table'].map(lambda x: len(x)==0)].index)

Парсим таблицу метаданных:

In [7]:
df['table'] = df['table'].map(lambda x: parse_table(x[0]))
df = df.join(df['table'].apply(pd.Series)).drop('table', axis=1)

Фильтруем только статьи с вынесением приговора:

In [8]:
only_guilty = lambda text: any([
    phrase in re.sub(r"\s+", "", text.lower(), flags=re.UNICODE) for phrase in ['судприговорил', 'приговорил']
])
df = df[df['body'].map(only_guilty)].reset_index(drop=True)

Очищаем текст от элементов анонимизации (напр. `дд.мм.гггг` или `<адрес>`)

In [9]:
filter_pattern = re.compile(r'<.*?>|дд\.мм\.гггг|-|\*')
filter_and_lower = lambda s: re.sub(filter_pattern, '', s.lower()).strip()

In [10]:
df['body'] = df['body'].map(filter_and_lower)

Разделяем судебные решения словами `установил` <содержательная часть> `приговорил` <решение суда>

In [11]:
def split_body(text, split_pattern):
    def fix_keywords(text):
        text = text.lower()
        text = re.sub('у\s?с\s?т\s?а\s?н\s?о\s?в\s?и\s?л', 'установил', text)
        text = re.sub('п\s?р\s?и\s?г\s?о\s?в\s?о\s?р\s?и\s?л', 'приговорил', text)  
        return text

    return re.split(split_pattern, fix_keywords(text))

In [12]:
df['body'] = df['body'].map(lambda doc: split_body(doc, re.compile(r'приговорил|установил')))

In [13]:
df = df.drop(df[df['body'].map(len) != 3].index)
df = df.reset_index(drop=True)

In [14]:
df['intro'], df['case'], df['result'] =  df['body'].str

Из решения суда выделяем номера статей кодекса:

In [15]:
pattern = re.compile(r'(статьей|ст\.|статьи)\s*(\d+\.?\d*)')
select_cc = lambda text: list(map(lambda x: float(x[1]), re.findall(pattern, text)))
df['labels'] = df['result'].map(select_cc)

In [16]:
df = df[df['labels'].map(len) != 0]

Оставляем только те статьи кодекса, которые мы знаем:

In [17]:
only_known = lambda arr: list(filter(lambda x: x in code['number'].tolist(), arr))
df['labels'] = df['labels'].map(only_known)

In [18]:
df = df[df['labels'].map(len) != 0]

In [19]:
df = df.rename(columns={
    'Категория':'category',
    'Дата':'date',
    'Регион':'region',
    'Суд':'court',
    'Судья': 'judge',
})
df = df[['title', 'date', 'category', 'region', 'court', 'judge', 'intro', 'case', 'result', 'labels']]

In [20]:
print('Количество судебных решений ', df.shape[0])

Количество судебных решений  17142


In [21]:
m = Mystem(entire_input=False)
ru_stopwords= get_stop_words('ru')

In [22]:
def lemm_and_filter(text):
    lemmas = m.lemmatize(text)
    filter_cond = lambda word: word not in ru_stopwords and len(word) > 3
    return list(filter(filter_cond, lemmas))

In [23]:
df['lemmas'] = df['case'].map(lemm_and_filter)

In [24]:
df.to_csv('tmp/judicial_orders.csv', index=False)